In [1]:
import pandas as pd
import warnings
import pymongo
from datetime import datetime
# pd.set_option('display.max_rows', None)

pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore', category=RuntimeWarning)

# Setup MongoDB connection (local)
mongo_host = "localhost"
mongo_port = 27017
mongo_user = "admin"
mongo_password = "password"
auth_db = "admin"
client_mongo = pymongo.MongoClient(
    host=mongo_host,
    port=mongo_port,
    username=mongo_user,
    password=mongo_password,
    authSource=auth_db
)

db_mongo_denormalization = client_mongo.get_database("denormalization")
db_mongo_datalake = client_mongo.get_database("datalake")

# GET brent_eur_liter (daily data)
collection_mongo = db_mongo_datalake.get_collection("stockmarket_brent_eur_liter")
cursor = collection_mongo.find({},{"_id": 0, "Date": 1, "Close": 1})
df_brent_eur_liter = pd.DataFrame(list(cursor))
df_brent_eur_liter = df_brent_eur_liter.rename(columns={'Close': 'BRENT_eur_liter'})
df_brent_eur_liter['Date'] = pd.to_datetime(df_brent_eur_liter['Date'])
print("df_brent_eur_liter\n", df_brent_eur_liter.head())


# GET htt_gas_eur_liter (weekly data)
collection_mongo = db_mongo_denormalization.get_collection("htt_gas_eur_liter")
cursor = collection_mongo.find({},{"_id": 0, "Date": 1, "htt_GAZOLE_eur_liter": 1,
                                   "htt_SP95_eur_liter": 1, "htt_E10_eur_liter": 1, })
df_htt_gas_eur_liter = pd.DataFrame(list(cursor))
df_htt_gas_eur_liter['Date'] = pd.to_datetime(df_htt_gas_eur_liter['Date'])
# Convert weekly data to daily data
df_htt_gas_eur_liter = df_htt_gas_eur_liter.set_index('Date')
df_htt_gas_eur_liter = df_htt_gas_eur_liter.resample('D').ffill()
df_htt_gas_eur_liter = df_htt_gas_eur_liter.reset_index()
print("df_htt_gas_eur_liter\n", df_htt_gas_eur_liter.head())


# GET refining_margin_brent_eur (monthly data)
collection_mongo = db_mongo_denormalization.get_collection("refining_margin_brent_eur")
cursor = collection_mongo.find({},{"_id": 0, "Date_monthly": 1, "refining_margin_brent_eur_liter": 1})
df_refining_margin_brent_eur_liter = pd.DataFrame(list(cursor))
df_refining_margin_brent_eur_liter = df_refining_margin_brent_eur_liter.rename(columns={'Date_monthly': 'Date'})
df_refining_margin_brent_eur_liter['Date'] = pd.to_datetime(df_refining_margin_brent_eur_liter['Date'])
# Convert monthly data to daily data
df_refining_margin_brent_eur_liter = df_refining_margin_brent_eur_liter.set_index('Date')
df_refining_margin_brent_eur_liter = df_refining_margin_brent_eur_liter.resample('D').ffill()
df_refining_margin_brent_eur_liter = df_refining_margin_brent_eur_liter.reset_index()
print("df_refining_margin_brent_eur_liter\n", df_refining_margin_brent_eur_liter.head())

# --- Merge the 3 df ---
df_merged = pd.merge(df_brent_eur_liter, df_htt_gas_eur_liter, on='Date', how='left')
df_merged = pd.merge(df_merged, df_refining_margin_brent_eur_liter, on='Date', how='left')
df_merged = df_merged[df_merged['refining_margin_brent_eur_liter'].notna()].reset_index(drop=True)
print("df_merged\n", df_merged.head())

# ------------- GET calcul for transportation_fees  -----------
# HTT_gas_eur_liter = Brent_eur_liter + refining_margin_brent_eur_liter + transportation_fees_eur_liter
# so 
# transportation_fees = gas_HTT_liter - ( Brent_eur_liter + refining_margin_brent_eur_liter )

# --- for GAZOLE ----
df_fees_gazole_eur_liter = pd.DataFrame()
df_fees_gazole_eur_liter['Date'] = df_merged['Date']
df_fees_gazole_eur_liter['Transportation_fees'] = (
    df_merged['htt_GAZOLE_eur_liter'] - ( df_merged['BRENT_eur_liter'] + df_merged['refining_margin_brent_eur_liter'] )
)
# extract Month and group by Month for average
df_fees_gazole_eur_liter["Year"] = df_fees_gazole_eur_liter["Date"].dt.year
df_fees_gazole_eur_liter["Month"] = df_fees_gazole_eur_liter["Date"].dt.month
df_monthly_avg_fees_gazole = df_fees_gazole_eur_liter.groupby(["Year", "Month"])["Transportation_fees"].mean().reset_index()
df_monthly_avg_fees_gazole["Transportation_fees"] = df_monthly_avg_fees_gazole["Transportation_fees"]*(2/3)
df_monthly_avg_fees_gazole["Transportation_fees"] = df_monthly_avg_fees_gazole["Transportation_fees"].round(5)
# replace negative margin value to zero
df_monthly_avg_fees_gazole['Transportation_fees'] = df_monthly_avg_fees_gazole['Transportation_fees'].clip(lower=0)
df_monthly_avg_fees_gazole['Date'] = pd.to_datetime(df_monthly_avg_fees_gazole[['Year', 'Month']].assign(DAY=1))
df_monthly_avg_fees_gazole = df_monthly_avg_fees_gazole[['Date', 'Transportation_fees']]
df_monthly_avg_fees_gazole = df_monthly_avg_fees_gazole.rename(columns={'Transportation_fees': 'calc_transp_fees_GAZOLE_eur_liter'})
# print('df_monthly_avg_fees_gazole\n', df_monthly_avg_fees_gazole)


# --- for SP95 ----
df_fees_sp95_eur_liter = pd.DataFrame()
df_fees_sp95_eur_liter['Date'] = df_merged['Date']
df_fees_sp95_eur_liter['Transportation_fees'] = (
    df_merged['htt_SP95_eur_liter'] - ( df_merged['BRENT_eur_liter'] + df_merged['refining_margin_brent_eur_liter'] )
)
# extract Month and group by Month for average
df_fees_sp95_eur_liter["Year"] = df_fees_sp95_eur_liter["Date"].dt.year
df_fees_sp95_eur_liter["Month"] = df_fees_sp95_eur_liter["Date"].dt.month
df_monthly_avg_fees_sp95 = df_fees_sp95_eur_liter.groupby(["Year", "Month"])["Transportation_fees"].mean().reset_index()
df_monthly_avg_fees_sp95["Transportation_fees"] = df_monthly_avg_fees_sp95["Transportation_fees"]*(2/3)
df_monthly_avg_fees_sp95["Transportation_fees"] = df_monthly_avg_fees_sp95["Transportation_fees"].round(5)
# replace negative margin value to zero
df_monthly_avg_fees_sp95['Transportation_fees'] = df_monthly_avg_fees_sp95['Transportation_fees'].clip(lower=0)
df_monthly_avg_fees_sp95['Date'] = pd.to_datetime(df_monthly_avg_fees_sp95[['Year', 'Month']].assign(DAY=1))
df_monthly_avg_fees_sp95 = df_monthly_avg_fees_sp95[['Date', 'Transportation_fees']]
df_monthly_avg_fees_sp95 = df_monthly_avg_fees_sp95.rename(columns={'Transportation_fees': 'calc_transp_fees_SP95_eur_liter'})
# print('df_monthly_avg_fees_sp95\n', df_monthly_avg_fees_sp95)


# --- for E10 ----
df_fees_e10_eur_liter = pd.DataFrame()
df_fees_e10_eur_liter['Date'] = df_merged['Date']
df_fees_e10_eur_liter['Transportation_fees'] = (
    df_merged['htt_E10_eur_liter'] - ( df_merged['BRENT_eur_liter'] + df_merged['refining_margin_brent_eur_liter'] )
)
# extract Month and group by Month for average
df_fees_e10_eur_liter["Year"] = df_fees_e10_eur_liter["Date"].dt.year
df_fees_e10_eur_liter["Month"] = df_fees_e10_eur_liter["Date"].dt.month
df_monthly_avg_fees_e10 = df_fees_e10_eur_liter.groupby(["Year", "Month"])["Transportation_fees"].mean().reset_index()
df_monthly_avg_fees_e10["Transportation_fees"] = df_monthly_avg_fees_e10["Transportation_fees"]*(2/3)
df_monthly_avg_fees_e10["Transportation_fees"] = df_monthly_avg_fees_e10["Transportation_fees"].round(5)
# replace negative margin value to zero
df_monthly_avg_fees_e10['Transportation_fees'] = df_monthly_avg_fees_e10['Transportation_fees'].clip(lower=0)
df_monthly_avg_fees_e10['Date'] = pd.to_datetime(df_monthly_avg_fees_e10[['Year', 'Month']].assign(DAY=1))
df_monthly_avg_fees_e10 = df_monthly_avg_fees_e10[['Date', 'Transportation_fees']]
df_monthly_avg_fees_e10 = df_monthly_avg_fees_e10.rename(columns={'Transportation_fees': 'calc_transp_fees_E10_eur_liter'})
# print('df_monthly_avg_fees_e10\n', df_monthly_avg_fees_e10)


df_calc_transp_fees_gas_eur_liter = pd.merge(df_monthly_avg_fees_gazole, df_monthly_avg_fees_sp95, on='Date', how='left')
df_calc_transp_fees_gas_eur_liter = pd.merge(df_calc_transp_fees_gas_eur_liter, df_monthly_avg_fees_e10, on='Date', how='left')
print('df_calc_transp_fees_gas_eur_liter\n', df_calc_transp_fees_gas_eur_liter)


# ----- Push to MongoDB -----
db_mongo_denormalization.drop_collection("calc_transp_fees_gas_eur_liter")
collection_mongo = db_mongo_denormalization.get_collection("calc_transp_fees_gas_eur_liter")
collection_mongo.create_index([("Date", pymongo.ASCENDING)])

records = df_calc_transp_fees_gas_eur_liter.to_dict(orient="records")
collection_mongo.insert_many(records)
print("correctly loaded df_calc_transp_fees_gas_eur_liter to denormalized collection MongoDB")

df_brent_eur_liter
         Date  BRENT_eur_liter
0 2007-07-30            0.347
1 2007-07-31            0.354
2 2007-08-01            0.347
3 2007-08-02            0.348
4 2007-08-03            0.341
df_htt_gas_eur_liter
         Date  htt_GAZOLE_eur_liter  htt_SP95_eur_liter  htt_E10_eur_liter
0 1985-01-04                0.3583                 NaN                NaN
1 1985-01-05                0.3583                 NaN                NaN
2 1985-01-06                0.3583                 NaN                NaN
3 1985-01-07                0.3583                 NaN                NaN
4 1985-01-08                0.3583                 NaN                NaN
df_refining_margin_brent_eur_liter
         Date  refining_margin_brent_eur_liter
0 2015-01-01                          0.03769
1 2015-01-02                          0.03769
2 2015-01-03                          0.03769
3 2015-01-04                          0.03769
4 2015-01-05                          0.03769
df_merged
         Dat

In [2]:
# df_annual_avg Diesel (2/3)
#  Year  Transportation_fees
# 0   2015             0.090782
# 1   2016             0.087676
# 2   2017             0.097743
# 3   2018             0.121216
# 4   2019             0.137487
# 5   2020             0.129756
# 6   2021             0.128049
# 7   2022             0.167633
# 8   2023             0.233285
# 9   2024             0.201898
# 10  2025             0.206323


# df_annual_avg SP95 (3/5)
#      Year  Transportation_fees
# 0   2015             0.124841
# 1   2016             0.128188
# 2   2017             0.123388
# 3   2018             0.125941
# 4   2019             0.143299
# 5   2020             0.153399
# 6   2021             0.169223
# 7   2022             0.110590
# 8   2023             0.271482
# 9   2024             0.260485
# 10  2025             0.256651

# df_annual_avg E10 (3/5)
#      Year  Transportation_fees
# 0   2015             0.119567
# 1   2016             0.128962
# 2   2017             0.123462
# 3   2018             0.126322
# 4   2019             0.142968
# 5   2020             0.158004
# 6   2021             0.169293
# 7   2022             0.097051
# 8   2023             0.270130
# 9   2024             0.251545
# 10  2025             0.245536